# ASSIGNMENT 1

In [53]:
!pip install wikipedia-api


In [54]:
import wikipediaapi
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import NaiveBayesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from nltk import classify
from nltk import NaiveBayesClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [55]:
# Define lists of terms
location_terms = ["region", "area", "terrain", "nation", "town"]
non_location_terms = ["innovation", "knowledge", "machine learning", "computing"]

def get_wiki_content_with_terms(topic, terms_list):
    # Initialize Wikipedia API object
    wiki_api = wikipediaapi.Wikipedia('en', extract_format=wikipediaapi.ExtractFormat.WIKI, headers={'User-Agent': 'Farid_Tavakkolinia'})

    # Get the Wikipedia page for the given topic
    wiki_page = wiki_api.page(topic)

    if wiki_page.exists():
        # Obtain the text content of the page
        content = wiki_page.text

        # Find terms within the text
        found_terms = [term for term in terms_list if term.lower() in content.lower()]

        return content, found_terms
    else:
        print(f"The page for '{topic}' does not exist.")
        return None, None

# Retrieve Wikipedia page content with terms
topic = "Machine learning"
content, found_terms = get_wiki_content_with_terms(topic, non_location_terms)

if content:
    print(f"Content for '{topic}':")
    print(content[:500])  # Display the first 500 characters of the content
    print("Found Terms:", found_terms)

def process_text_with_terms(text, terms, stopwords_set=None, text_stemmer=None, text_lemmatizer=None):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Use provided stopwords set or default to NLTK stop words
    stopwords_set = stopwords_set or set(stopwords.words('english'))

    # Filter out stopwords
    filtered_tokens = [token for token in tokens if token.lower() not in stopwords_set]

    # Use provided stemmer or default to Porter Stemmer
    text_stemmer = text_stemmer or PorterStemmer()
    stemmed_tokens = [text_stemmer.stem(token) for token in filtered_tokens]

    # Use provided lemmatizer or default to WordNet Lemmatizer
    text_lemmatizer = text_lemmatizer or WordNetLemmatizer()
    lemmatized_tokens = [text_lemmatizer.lemmatize(token) for token in stemmed_tokens]

    # Reconstruct the preprocessed text
    processed_text = ' '.join(lemmatized_tokens)

    # Append terms to the processed text
    processed_text += ' ' + ' '.join(terms)

    return processed_text

# Use Bag of Words without preprocessing
bow_text = content.lower()
bow_features = {word: True for word in word_tokenize(bow_text)}

# Use Snowball stemmer and custom stopwords
snowball_stemmer = SnowballStemmer('english')
processed_text_with_snowball = process_text_with_terms(content, found_terms, stopwords_set=set(stopwords.words('english')), text_stemmer=snowball_stemmer)

# Use WordNet lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
processed_text_with_lemmatizer = process_text_with_terms(content, found_terms, stopwords_set=set(stopwords.words('english')), text_lemmatizer=wordnet_lemmatizer)

# Print the results
print("\nBag of Words without preprocessing:")
print(bow_features)

print("\nProcessed Text with Snowball stopwords and stemmer:")
print(processed_text_with_snowball[:500])

print("\nProcessed Text with WordNet Lemmatizer:")
print(processed_text_with_lemmatizer[:500])

Content for 'Machine learning':
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Recently, artificial neural networks have been able to surpass many previous approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and med
Found Terms: ['knowledge', 'machine learning', 'computing']

Bag of Words without preprocessing:
{'machine': True, 'learning': True, '(': True, 'ml': True, ')': True, 'is': True, 'a': True, 'field': True, 'of': True, 'study': True, 'in': True, 'artificial': True, 'intelligence': True, 'concerned': True, 'with': True, 'the': True, 'development': True, 'and': True, 'statistical': True, 'algorithms': True, 'that': True, 'can': True, 'learn': True, 'from': True, 'dat

## Naive Bayes without Pre-processing:

In [56]:
# 'geographic' and 'non-geographic' are classes
geographic_class = "location"
non_geographic_class = "non-location"

# Training data for Naive Bayes on Bag of Words without pre-processing
training_data_bow_raw = [
    (get_wiki_content_with_terms("Florence", location_terms)[0].lower(), geographic_class),
    (get_wiki_content_with_terms("Machine learning", non_location_terms)[0].lower(), non_geographic_class),
]

# Tokenizer function for raw text
def tokenize_raw_text_bow(text):
    return word_tokenize(text)

# Feature extraction function for raw text
def extract_features_raw_bow(text):
    return {word: True for word in tokenize_raw_text_bow(text)}

# Prepare the training set without pre-processing for Bag of Words
training_set_bow_raw = [(extract_features_raw_bow(text), label) for (text, label) in training_data_bow_raw]

# Train the Naive Bayes classifier without pre-processing for Bag of Words
nb_classifier_bow_raw = NaiveBayesClassifier.train(training_set_bow_raw)

# Usage without pre-processing for Bag of Words
test_text_bow_raw = get_wiki_content_with_terms("Florence", location_terms)[0].lower()
test_features_bow_raw = extract_features_raw_bow(test_text_bow_raw)
classification_bow_raw = nb_classifier_bow_raw.classify(test_features_bow_raw)

print(f"\nPredicted class for the test text without pre-processing for Bag of Words: {classification_bow_raw}")

test_text_bow_raw = get_wiki_content_with_terms("Data Science", non_location_terms)[0].lower()
test_features_bow_raw = extract_features_raw_bow(test_text_bow_raw)
classification_bow_raw = nb_classifier_bow_raw.classify(test_features_bow_raw)

print(f"Predicted class for the test text without pre-processing for Bag of Words: {classification_bow_raw}")


Predicted class for the test text without pre-processing for Bag of Words: location
Predicted class for the test text without pre-processing for Bag of Words: non-location


## Naive Bayes with Pre-processing:


In [57]:
# Training data for Naive Bayes with Snowball stop words and Snowball Stemmer
training_data_bow_snowball = [
    (process_text_with_terms(get_wiki_content_with_terms("Florence", location_terms)[0], location_terms, stopwords_set=set(stopwords.words('english')), text_stemmer=snowball_stemmer).lower(), geographic_class),
    (process_text_with_terms(get_wiki_content_with_terms("Machine learning", non_location_terms)[0], non_location_terms, stopwords_set=set(stopwords.words('english')), text_stemmer=snowball_stemmer).lower(),
                                non_geographic_class),
]

# Tokenizer function for preprocessed text
def tokenize_preprocessed_text_bow(text):
    return word_tokenize(text)

# Feature extraction function for preprocessed text
def extract_features_preprocessed_bow(text):
    return {word: True for word in tokenize_preprocessed_text_bow(text)}

# Prepare the training set with Snowball stop words and Snowball Stemmer for Bag of Words
training_set_bow_snowball = [(extract_features_preprocessed_bow(text), label) for (text, label) in training_data_bow_snowball]

# Train the Naive Bayes classifier with Snowball stop words and Snowball Stemmer for Bag of Words
nb_classifier_bow_snowball = NaiveBayesClassifier.train(training_set_bow_snowball)

# Usage with Snowball stop words and Snowball Stemmer for Bag of Words
test_text_bow_snowball = process_text_with_terms(get_wiki_content_with_terms("Florence", location_terms)[0], location_terms, stopwords_set=set(stopwords.words('english')), text_stemmer=snowball_stemmer).lower()
test_features_bow_snowball = extract_features_preprocessed_bow(test_text_bow_snowball)
classification_bow_snowball = nb_classifier_bow_snowball.classify(test_features_bow_snowball)

print(f"\nPredicted class for the preprocessed test text with Snowball stop words and Snowball Stemmer for Bag of Words: {classification_bow_snowball}")

test_text_bow_snowball = process_text_with_terms(get_wiki_content_with_terms("Data Science", non_location_terms)[0], non_location_terms, stopwords_set=set(stopwords.words('english')),
                                                 text_stemmer=snowball_stemmer).lower()
test_features_bow_snowball = extract_features_preprocessed_bow(test_text_bow_snowball)
classification_bow_snowball = nb_classifier_bow_snowball.classify(test_features_bow_snowball)

print(f"Predicted class for the preprocessed test text with Snowball stop words and Snowball Stemmer for Bag of Words: {classification_bow_snowball}")


Predicted class for the preprocessed test text with Snowball stop words and Snowball Stemmer for Bag of Words: location
Predicted class for the preprocessed test text with Snowball stop words and Snowball Stemmer for Bag of Words: non-location


## Logistic Regression

### Training data for Logistic Regression with pre-processing from Naive Bayes


In [58]:
### Training data for Logistic Regression with pre-processing from Naive Bayes


# Training data for Logistic Regression with pre-processing from Naive Bayes
logistic_train_data_preprocessed = [
    (process_text_with_terms(get_wiki_content_with_terms("Florence", location_terms)[0], location_terms, stopwords_set=set(stopwords.words('english')), text_stemmer=snowball_stemmer).lower(),
                            geographic_class),
    (process_text_with_terms(get_wiki_content_with_terms("Machine learning", non_location_terms)[0], non_location_terms, stopwords_set=set(stopwords.words('english')), text_stemmer=snowball_stemmer).lower(),
                            non_geographic_class),
]

# Tokenizer function for preprocessed text (same as used for Naive Bayes)
def tokenize_preprocessed_logistic_text(text):
    return word_tokenize(text)

# Feature extraction function for preprocessed text (same as used for Naive Bayes)
def extract_preprocessed_logistic_features(text):
    return ' '.join(tokenize_preprocessed_logistic_text(text))

# Prepare the training set for Logistic Regression with pre-processing
logistic_train_set_preprocessed = [(extract_preprocessed_logistic_features(text), label) for (text, label) in logistic_train_data_preprocessed]

# Create TF-IDF vectors from the training set for Logistic Regression with pre-processing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Separate features (X) and labels (y) for Logistic Regression with pre-processing
X_train_logistic_preprocessed = [text for (text, _) in logistic_train_set_preprocessed]
y_train_logistic_preprocessed = [label for (_, label) in logistic_train_set_preprocessed]

# Create TF-IDF vectors from the training set for Logistic Regression with pre-processing
tfidf_vectorizer_logistic_preprocessed = TfidfVectorizer()
X_train_tfidf_logistic_preprocessed = tfidf_vectorizer_logistic_preprocessed.fit_transform(X_train_logistic_preprocessed)

# Train the Logistic Regression classifier with pre-processing
logistic_regressor_preprocessed = LogisticRegression()
logistic_regressor_preprocessed.fit(X_train_tfidf_logistic_preprocessed, y_train_logistic_preprocessed)

# Usage for Logistic Regression with pre-processing
test_text_logistic_preprocessed = process_text_with_terms(get_wiki_content_with_terms("Florence", location_terms)[0], location_terms, stopwords_set=set(stopwords.words('english')),
                                                          text_stemmer=snowball_stemmer).lower()
test_features_logistic_preprocessed = tfidf_vectorizer_logistic_preprocessed.transform([extract_preprocessed_logistic_features(test_text_logistic_preprocessed)])
predicted_class_logistic_preprocessed = logistic_regressor_preprocessed.predict(test_features_logistic_preprocessed)

print(f"\nPredicted class for the test text using Logistic Regression with pre-processing: {predicted_class_logistic_preprocessed[0]}")

test_text_logistic_preprocessed = process_text_with_terms(get_wiki_content_with_terms("Data Science", non_location_terms)[0], non_location_terms, stopwords_set=set(stopwords.words('english')),
                                                          text_stemmer=snowball_stemmer).lower()
test_features_logistic_preprocessed = tfidf_vectorizer_logistic_preprocessed.transform([extract_preprocessed_logistic_features(test_text_logistic_preprocessed)])
predicted_class_logistic_preprocessed = logistic_regressor_preprocessed.predict(test_features_logistic_preprocessed)

print(f"Predicted class for the test text using Logistic Regression with pre-processing: {predicted_class_logistic_preprocessed[0]}")

# Training data for Logistic Regression without pre-processing
logistic_train_data_raw = [
    (get_wiki_content_with_terms("Florence", location_terms)[0].lower(), geographic_class),
    (get_wiki_content_with_terms("Machine learning", non_location_terms)[0].lower(), non_geographic_class),
]

# Tokenizer function for raw text (same as used for Naive Bayes)
def tokenize_raw_logistic_text(text):
    return word_tokenize(text)

# Feature extraction function for raw text (same as used for Naive Bayes)
def extract_raw_logistic_features(text):
    return ' '.join(tokenize_raw_logistic_text(text))

# Prepare the training set for Logistic Regression without pre-processing
logistic_train_set_raw = [(extract_raw_logistic_features(text), label) for (text, label) in logistic_train_data_raw]

# Create TF-IDF vectors from the training set for Logistic Regression without pre-processing
X_train_logistic_raw = [text for (text, _) in logistic_train_set_raw]
y_train_logistic_raw = [label for (_, label) in logistic_train_set_raw]

tfidf_vectorizer_logistic_raw = TfidfVectorizer()
X_train_tfidf_logistic_raw = tfidf_vectorizer_logistic_raw.fit_transform(X_train_logistic_raw)

# Train the Logistic Regression classifier without pre-processing
logistic_regressor_raw = LogisticRegression()
logistic_regressor_raw.fit(X_train_tfidf_logistic_raw, y_train_logistic_raw)


Predicted class for the test text using Logistic Regression with pre-processing: location
Predicted class for the test text using Logistic Regression with pre-processing: non-location


LogisticRegression()

### Usage for Logistic Regression without pre-processing


In [59]:
test_text_logistic_raw = get_wiki_content_with_terms("Florence", location_terms)[0].lower()
test_features_logistic_raw = tfidf_vectorizer_logistic_raw.transform([extract_raw_logistic_features(test_text_logistic_raw)])
predicted_class_logistic_raw = logistic_regressor_raw.predict(test_features_logistic_raw)

print(f"\nPredicted class for the test text using Logistic Regression without pre-processing: {predicted_class_logistic_raw[0]}")

test_text_logistic_raw = get_wiki_content_with_terms("Data Science", non_location_terms)[0].lower()
test_features_logistic_raw = tfidf_vectorizer_logistic_raw.transform([extract_raw_logistic_features(test_text_logistic_raw)])
predicted_class_logistic_raw = logistic_regressor_raw.predict(test_features_logistic_raw)

print(f"Predicted class for the test text using Logistic Regression without pre-processing: {predicted_class_logistic_raw[0]}")


Predicted class for the test text using Logistic Regression without pre-processing: location
Predicted class for the test text using Logistic Regression without pre-processing: non-location


# ASSIGNMENT 2

In [60]:
# Install Faker library for generating dummy text
!pip install faker


In [61]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from faker import Faker

In [62]:
# Download NLTK's Punkt tokenizer models
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [63]:
CONTEXT_WINDOW_LIMIT = 4000  # The maximum number of tokens allowed in the context window

def measure_length(text):
    """
    Measure the length of the text in terms of the number of tokens.
    """
    tokens = word_tokenize(text)
    return len(tokens)

def proportional_length(length1, length2, target_length):
    """
    Compute the target lengths for the summaries proportionally based on the original lengths.
    """
    proportion = target_length / (length1 + length2)
    return int(length1 * proportion), int(length2 * proportion)

def slice_document(text, max_tokens):
    """
    Slice the document into chunks that fit within the context window limit.
    """
    tokens = word_tokenize(text)
    for i in range(0, len(tokens), max_tokens):
        yield ' '.join(tokens[i:i + max_tokens])

def summarize_text(text, target_length=None):
    """
    Summarize the text to the target length in terms of tokens. If no target length is provided,
    return the text as it is.
    """
    tokens = word_tokenize(text)
    if target_length and len(tokens) > target_length:
        return ' '.join(tokens[:target_length])
    return text

def collate_summaries(summaries):
    """
    Collate all the summaries into a single summary.
    """
    return ' '.join(summaries)

def hierarchical_summarization(document, max_tokens):
    """
    Perform hierarchical summarization on the document to ensure the summary fits
    within the context window limit.
    """
    summaries = []
    # Slice the document and summarize each slice
    for slice in slice_document(document, max_tokens):
        summaries.append(summarize_text(slice, max_tokens))

    # Collate all summaries into a single summary
    combined_summary = collate_summaries(summaries)

    # If the combined summary exceeds the context window, further summarize it
    while measure_length(combined_summary) > CONTEXT_WINDOW_LIMIT:
        combined_summary = summarize_text(combined_summary, CONTEXT_WINDOW_LIMIT // 2)

    return combined_summary

def save_document(summary, filename):
    """
    Save the summarized document to a file.
    """
    with open(filename, 'w') as file:
        file.write(summary)

In [64]:
# Initialize Faker for generating dummy documents
fake = Faker()

# Generate two large documents with dummy text
document1 = " ".join(fake.paragraphs(nb=200))  # First document with 200 paragraphs
document2 = " ".join(fake.paragraphs(nb=500))  # Second document with 500 paragraphs

# Measure the lengths of the documents in tokens
length1 = measure_length(document1)
length2 = measure_length(document2)

# Compute the target lengths proportionally based on the total target length (4000 tokens)
target_length1, target_length2 = proportional_length(length1, length2, CONTEXT_WINDOW_LIMIT)

# Perform hierarchical summarization on both documents
summary1 = hierarchical_summarization(document1, target_length1)
summary2 = hierarchical_summarization(document2, target_length2)

# Save the summarized documents to files
save_document(summary1, 'summary1.txt')
save_document(summary2, 'summary2.txt')

print("Summarization complete and saved.")

Summarization complete and saved.


In [65]:
def display_summary(filename, num_sentences=5):
    """
    Read the saved summary from the file and display the first few sentences.
    """
    with open(filename, 'r') as file:
        content = file.read()
        sentences = sent_tokenize(content)
        # Display the first few sentences of the summary
        print(f"--- {filename} ---")
        print(' '.join(sentences[:num_sentences]))
        print("\n")

# Display the first few sentences of the saved summaries
display_summary('summary1.txt')
display_summary('summary2.txt')

--- summary1.txt ---
Summer five clearly career oil also . Quickly plant nothing end . Treat knowledge say six shoulder . Ability trouble southern view everyone whatever garden . Dinner personal democratic consumer her house .


--- summary2.txt ---
Huge true phone share always . Whom and order similar leader why thousand environment . Himself easy entire prove leave local they . Discussion group pretty receive . Main middle international .


